## Heilbrigðislíkan
Smá byrjun

In [5]:
import simpy as sp
import streamlit as st
import numpy as np
import math
import random
import time
import os
import signal
from multiprocessing import Process
import pandas as pd
import matplotlib.pyplot as plt

In [14]:
TIMI = 0
N = 1000
T = 1

In [7]:
class Manneskja:
    def __init__(self,aldur):
        self.aldur = aldur
        if self.aldur < 65:
            self.prob = 0.02
        else:
            self.prob = 0.1
    def MtoP(self):
        p = Patient(self.aldur,"móttaka")
        return p

class Patient:
    def __init__(self,aldur,place):
        self.aldur = aldur
        self.place = place
        if place == "skurðaaðgerð":
            self.p_death = 0.1
            self.p_surgery = 0.0
            self.p_mottaka = 0.9
        if place == "móttaka":
            self.p_death = 0.0
            self.p_home = 0.7
            self.p_surgery = 0.3
    def PtoM(self):
        m = Manneskja(self.aldur)
        return m
    
class Spitali:
    def __init__(self,cap):
        self.cap = cap
        self.amount = 0
    def addP(self):
        if self.amount == self.cap:
            raise Exception("Nú er spítalinn fullur það tók")
        self.amount += 1
    def removeP(self):
        self.amount -= 1

In [16]:
#sliders o.fl.
with st.expander("Breyta hlutum"):
    h_A = st.slider("Hlutfall aldraðra",value = 0.3,step = 0.02)

h_U = 1-h_A

In [17]:
labels = "Aldraðir","Ungir"
sizes = [h_A*N,h_U*N]

fig1,ax1 = plt.subplots()
ax1.pie(sizes,labels = labels)

st.pyplot(fig1)

DeltaGenerator()

In [20]:
def sim():
    n = N
    timi = TIMI
    t = T
    P = []            # fjöldi sjúklinga á spítala
    M = []            # fjöldi manneskja ekki á spítala
    S = Spitali(500)  # spítali með capacity 500
    d = {0:0}
    df = pd.DataFrame(d,index=[0])
    chart = st.line_chart(df)
    for p in range(0, int(h_A*n)):
        age = random.randint(65, 99)
        pi = Manneskja(age)
        M.append(pi)
    for p in range(int(h_A*n), n):
        age = random.randint(1, 64)
        pi = Manneskja(age)
        M.append(pi)
    STOP = 10000
    while(timi < STOP):
        timi += t
        for m in M:
            r = random.random()
            if r < m.prob:
                S.addP()
                pi = m.MtoP()
                M.remove(m)
                P.append(pi)
        for p in P:
            r = random.random()
            if p.place == "móttaka":
                if r < p.p_death:
                    P.remove(p)
                    S.removeP()
                if r >= p.p_death and r <= p.p_surgery:
                    P.remove(p)
                    S.removeP()
                    m = p.PtoM()
                    M.append(m)
                else:
                    p.place = "skurðaaðgerð"
            if p.place == "skurðaaðgerð":
                if r < p.p_surgery:
                    p.place = "skurðaaðgerð"
                if r >= p.p_surgery and r <= p.p_death:
                    P.remove(p)
                    S.removeP()
                else:
                    p.place = "móttaka"
        d = {timi: S.amount}
        df = pd.DataFrame(d,index=[0])
        chart.add_rows(df.values.tolist())
        time.sleep(0.1)
        print("fjöldi fólks á spítala á degi",timi,"er",S.amount)

In [19]:
st.sidebar.title("Controls")
start = st.sidebar.button("Start")
stop = st.sidebar.button("Stop")

if start:
    if not st.experimental_get_query_params():
        p = Process(target=sim)
        p.run()
        st.experimental_set_query_params(pid=p.pid)
        st.write("Started process with pid:", p.pid)
    else:
        st.write("Process is already running.")

if stop:
    timi = 10000
    query_params = st.experimental_get_query_params()
    pid = query_params.get("pid", None)
    if pid:
        os.kill(int(pid[0]), signal.SIGTERM)
        st.experimental_set_query_params()
        st.write("Stopped process with pid:", pid[0])
    else:
        st.write("No process running.")